[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sunfish256/fitbit-api-python/blob/main/app/update_token.ipynb)

In [ ]:
'''Module providing the function to read and write json files.'''
import json
from requests import Session

In [ ]:
# Settings
TOKEN_FILE_PATH = './fb_conf.json'

In [ ]:
def bearer_header():
    '''
    Header for Bearer Authentication.

    Returns:
        dict: {'Authorization':'Bearer ' + your-access-token}
    '''
    return {'Authorization': 'Bearer ' + conf['access_token']}


def refresh():
    '''
    Reacquire access_token and update conf.json.
    refresh_token is important because it is required for reacquisition.
    Called when access_token has expired.
    '''
    url = 'https://api.fitbit.com/oauth2/token'
    params = {
        'grant_type': 'refresh_token',
        'refresh_token': conf['refresh_token'],
        'client_id': conf['client_id'],
    }
    # Post. The body is application/x-www-form-urlencoded.
    res = session.post(url, data=params)
    # Parse
    res_data = res.json()
    # With error, output error messages
    if res_data.get('errors') is not None:
        emsg = res_data['errors'][0]
        print(f'{emsg}\n')
    # Without error, update the token file
    else:
        conf['access_token'] = res_data['access_token']
        conf['refresh_token'] = res_data['refresh_token']
        with open(TOKEN_FILE_PATH, 'w', encoding='utf-8') as f:
            json.dump(conf, f, indent=2)


def is_expired(res_object: dict):
    '''
    From the Response, check if the accesss-token has expired.
    True if it has expired, False if not. 8 hours is the lifetime for Fitbit API.

    Args:
        res_object (_type_): response.json()

    Returns:
        str or None: returns string if there is an error, otherwise returns a NoneType object
    '''
    errors = res_object.get('errors')
    if errors:
        for err in errors:
            etype = err.get('errorType')
            if etype == 'expired_token':
                print('Token has expired. Attempt to update token.\n')
            if etype == 'invalid_token':
                print('Token is invalid. Attempt to update token. If the problem persists,')
                print('check the token at https://dev.fitbit.com/apps.\n')
            else:
                print(etype)
    else:
        print('Current token is valid.\n')
    return errors


def request():
    '''
    Request heart rate data for the test.
    The access token expires in 8h, so when it expires,
    it is reacquired and the request is reexecuted.

    Returns:
        dict: response
    '''
    # Endpoint
    # url = 'https://api.fitbit.com/1/user/-/hrv/date/today/all.json'
    url = 'https://api.fitbit.com/1/user/-/activities/heart/date/today/1d.json'
    # Request
    res = session.get(url, headers=bearer_header())
    if is_expired(res.json()):
        # Update the token if it has expired
        refresh()
        # Update headers and re-run the request
        res = session.get(url, headers=bearer_header())
    return res.json()


if __name__ == '__main__':
    # Initialize session
    session = Session()
    # Read the authentication file
    with open(TOKEN_FILE_PATH, 'r', encoding='utf-8') as f:
        conf = json.load(f)
    # Request heart rate for the test
    res_hr = request()
    # Check if the response is correct
    if list(res_hr.keys()) == ['activities-heart', 'activities-heart-intraday']:
        print('The request succeeded.')
    else:
        print('The request failed.')